In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import os.path as path
import copy
from torch.utils.data.sampler import SubsetRandomSampler


from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer


# Settings 
## Dataset

In [4]:
crop_im_size = [186,250]
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
c.config['pose']['load'] = False
c.config['flow']['load'] = False
c.config['heatmaps']['load'] = False
c.config['dataset_output'] = {
#         'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
        'data': ["scenes"],
        'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)


loading configuration  trail2


## neural network

In [5]:
#DESIGN PARAMETERS FOR NEURAL NETWORK
NR_LSTM_UNITS = 2 
IMAGE_INPUT_SIZE_W = 640
IMAGE_INPUT_SIZE_H = 480

IMAGE_AFTER_CONV_SIZE_W = 18
IMAGE_AFTER_CONV_SIZE_H = 13
#for 3x3 kernels, n=num_layers: len_in = 2^n*len_out + sum[i=1..n](2^i)
#CONV_LAYER_LENGTH = 5

LSTM_IO_SIZE = 18*13
LSTM_HIDDEN_SIZE = 18*13

RGB_CHANNELS = 3
TIMESTEPS = 10 # size videos
BATCH_SIZE = 5 #until now just batch_size = 1

#USE RANDOM IMAGES TO SET UP WORKING EXAMPLE
class TEST_CNN_LSTM(nn.Module):
    def __init__(self):
        super(TEST_CNN_LSTM, self).__init__()
        self.avialable_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.conv1 = nn.Conv2d(3,6,3) #input 640x480
        self.pool1 = nn.MaxPool2d(2,2) #input 638x478 output 319x239
        self.conv2 = nn.Conv2d(6,16,3) # input 319x239 output 317x237
        self.pool2 = nn.MaxPool2d(2,2) # input 317x237 output 158x118
        self.conv3 = nn.Conv2d(16,6,3) # input 158x118 output 156x116
        self.pool3 = nn.MaxPool2d(2,2) # input 156x116 output 78x58
        self.conv4 = nn.Conv2d(6,3,3)  # input 78x58 output 76x56
        self.pool4 = nn.MaxPool2d(2,2) # input 76x56 output 39x29
        self.conv5 = nn.Conv2d(3,1,3)  # input 39x29 output 37x27
        self.pool5 = nn.MaxPool2d(2,2) #output 37x27 output 18x13
        self.lstm1 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# horizontal direction
        self.lstm2 = nn.LSTM(LSTM_IO_SIZE,
                            LSTM_HIDDEN_SIZE,
                            TIMESTEPS)# horizontal direction
        self.fc1 = nn.Linear(LSTM_IO_SIZE,120)
        self.fc2 = nn.Linear(120,20)
        self.fc3 = nn.Linear(20,3)
        
        #initialize hidden states of LSTM
        self.hidden = self.init_hidden()

        #print("Hidden:", _hidden)
    def init_hidden(self):
        return (torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device), 
                torch.randn(TIMESTEPS, BATCH_SIZE, LSTM_HIDDEN_SIZE).to(self.avialable_device))
    def forward(self,x):
#         print("Input list len:",len(x))
#         print("Input elemens size:", x[0].size())
#         batch_size = x[0].size()[0]

        x_arr = torch.zeros(TIMESTEPS,BATCH_SIZE,1,IMAGE_AFTER_CONV_SIZE_H,IMAGE_AFTER_CONV_SIZE_W).to(self.avialable_device)
        ## print("X arr size", x_arr.size())
        for i in range(TIMESTEPS):#parallel convolutions which are later concatenated for LSTM
            x_tmp_c1 = self.pool1(F.relu(self.conv1(x[i].float())))
            x_tmp_c2 = self.pool2(F.relu(self.conv2(x_tmp_c1)))
            x_tmp_c3 = self.pool3(F.relu(self.conv3(x_tmp_c2)))
            x_tmp_c4 = self.pool4(F.relu(self.conv4(x_tmp_c3)))
            x_tmp_c5 = self.pool5(F.relu(self.conv5(x_tmp_c4)))
            x_arr[i] = x_tmp_c5 # torch.squeeze(x_tmp_c5)
        
        x, hidden = self.lstm1(x_arr.view(TIMESTEPS,BATCH_SIZE,-1), self.hidden)
        x, hidden = self.lstm2(x, self.hidden)
        # the reshaping was taken from the documentation... and makes scense
        x = x.view(TIMESTEPS,BATCH_SIZE,LSTM_HIDDEN_SIZE) #output.view(seq_len, batch, num_dir*hidden_size)
#         x = torch.squeeze(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) 
        x = x.permute(1,2,0)
        return x
print("Class defined")

Class defined


In [6]:
# TODOS:
# Look for overfitting..
# use GPU: 300 samples. many epochs. learning rate maybe is too high.
# Tweak learning rate
# Increase the RNN size
# Maybe change the CNN
# Use patches. or OF
# Increase the dataset: 10 images 10peoplo+++ not too much too much variance. 
# FEEDBACK on friday..:) 

## Training-testing squeme
### hyperparameters



In [7]:
validation_split = 0.1
lr = .01 # reduce factos of 10 .. some epoch later.
momentum = 0.9
randomized_seed = 10
shuffle_dataset = False

### Test-Training splitting

In [8]:
#TRAINING
test_net = TEST_CNN_LSTM()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
test_net.to(device)
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
optimizer = optim.SGD(test_net.parameters(), lr=lr, momentum=momentum)

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(randomized_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

# 

cuda:0


In [9]:
# test_net.parameters
# pytorch_total_params = sum(p.numel() for p in test_net.parameters())
# pytorch_total_params


In [ ]:
### define testing function
def test_all_preds(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            scenes = [s.to(device) for s in inputs['scenes']]
            labels = labels.to(device)
            if not labels.size()[0] == BATCH_SIZE:
                # skip uncompleted batch size NN is fixed to BATCHSIZE
                continue
            outputs = model(scenes)
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')

### training over all predictions

In [ ]:
n_batches = len(train_loader)

#Time for printing
training_start_time = time.time()
    
print('Start training...')
for epoch in range(100): 
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        
        inputs, labels = batch
        scenes = [s.to(device) for s in inputs['scenes']]
        labels = labels.unsqueeze(-1).to(device)
        if not labels.size()[0] == BATCH_SIZE:
            # skip uncompleted batch size NN is fixed to BATCHSIZE
            continue
        optimizer.zero_grad() 
        outputs = test_net(scenes)
        outputs = outputs.unsqueeze(-1)
#         print("Out:", len(outputs), outputs.size())
#         print("Labels:", len(labels), labels.size())
        loss = criterion(outputs,labels.long() )
        loss.backward() 
        optimizer.step()
        
        #Print statistics
        # print(loss.data.item())
        running_loss += loss.data.item()
        total_train_loss += loss.data.item()
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss(mean): {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss/print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()
    # test after each epoch
    test_all_preds(test_net)
    print('total training loss for epoch {}: {:.6f}'.format(epoch+1, total_train_loss))

print('...Training finished')

Start training...
Epoch: 0


/home/ron/anaconda3/envs/gait_37/lib/python3.7/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Epoch 1, 11% 	 train_loss(mean): 2.24 took: 4.87s
Epoch 1, 23% 	 train_loss(mean): 2.21 took: 4.34s
Epoch 1, 35% 	 train_loss(mean): 2.22 took: 4.37s
Epoch 1, 47% 	 train_loss(mean): 2.22 took: 3.81s
Epoch 1, 58% 	 train_loss(mean): 2.19 took: 4.34s
Epoch 1, 70% 	 train_loss(mean): 2.19 took: 4.22s
Epoch 1, 82% 	 train_loss(mean): 2.22 took: 4.41s
Epoch 1, 94% 	 train_loss(mean): 2.17 took: 4.26s
Start testing...
Accuracy 47.00%
...testing finished
total training loss for epoch 1: 17.663430
Epoch: 1
Epoch 2, 11% 	 train_loss(mean): 2.13 took: 4.15s
Epoch 2, 23% 	 train_loss(mean): 2.19 took: 4.29s
Epoch 2, 35% 	 train_loss(mean): 2.13 took: 4.19s
Epoch 2, 47% 	 train_loss(mean): 2.19 took: 3.99s
Epoch 2, 58% 	 train_loss(mean): 2.16 took: 3.90s
Epoch 2, 70% 	 train_loss(mean): 2.06 took: 4.20s
Epoch 2, 82% 	 train_loss(mean): 2.26 took: 4.13s
Epoch 2, 94% 	 train_loss(mean): 2.01 took: 4.14s
Start testing...
Accuracy 47.50%
...testing finished
total training loss for epoch 2: 17.124510

Epoch 17, 23% 	 train_loss(mean): 1.99 took: 4.44s
Epoch 17, 35% 	 train_loss(mean): 1.93 took: 4.09s
Epoch 17, 47% 	 train_loss(mean): 2.13 took: 4.12s
Epoch 17, 58% 	 train_loss(mean): 2.14 took: 4.32s
Epoch 17, 70% 	 train_loss(mean): 1.89 took: 3.96s
Epoch 17, 82% 	 train_loss(mean): 1.89 took: 4.04s
Epoch 17, 94% 	 train_loss(mean): 1.88 took: 4.00s
Start testing...
Accuracy 47.62%
...testing finished
total training loss for epoch 17: 16.012955
Epoch: 17
Epoch 18, 11% 	 train_loss(mean): 1.85 took: 4.47s
Epoch 18, 23% 	 train_loss(mean): 1.89 took: 4.31s
Epoch 18, 35% 	 train_loss(mean): 2.14 took: 4.05s
Epoch 18, 47% 	 train_loss(mean): 2.08 took: 3.81s
Epoch 18, 58% 	 train_loss(mean): 2.35 took: 4.32s
Epoch 18, 70% 	 train_loss(mean): 2.13 took: 4.01s
Epoch 18, 82% 	 train_loss(mean): 1.70 took: 3.73s
Epoch 18, 94% 	 train_loss(mean): 1.83 took: 4.09s
Start testing...
Accuracy 47.00%
...testing finished
total training loss for epoch 18: 15.966124
Epoch: 18
Epoch 19, 11% 	 train

Epoch 33, 23% 	 train_loss(mean): 1.94 took: 4.05s
Epoch 33, 35% 	 train_loss(mean): 2.00 took: 5.53s
Epoch 33, 47% 	 train_loss(mean): 2.02 took: 4.22s
Epoch 33, 58% 	 train_loss(mean): 1.97 took: 4.27s
Epoch 33, 70% 	 train_loss(mean): 1.81 took: 3.98s
Epoch 33, 82% 	 train_loss(mean): 2.04 took: 4.25s
Epoch 33, 94% 	 train_loss(mean): 1.69 took: 3.95s
Start testing...
Accuracy 52.88%
...testing finished
total training loss for epoch 33: 15.536228
Epoch: 33
Epoch 34, 11% 	 train_loss(mean): 1.90 took: 5.17s
Epoch 34, 23% 	 train_loss(mean): 1.88 took: 3.90s
Epoch 34, 35% 	 train_loss(mean): 2.26 took: 4.57s
Epoch 34, 47% 	 train_loss(mean): 1.85 took: 4.17s
Epoch 34, 58% 	 train_loss(mean): 1.85 took: 4.19s
Epoch 34, 70% 	 train_loss(mean): 1.89 took: 4.25s
Epoch 34, 82% 	 train_loss(mean): 1.87 took: 3.78s
Epoch 34, 94% 	 train_loss(mean): 2.13 took: 4.18s
Start testing...
Accuracy 53.25%
...testing finished
total training loss for epoch 34: 15.641969
Epoch: 34
Epoch 35, 11% 	 train

Epoch 49, 23% 	 train_loss(mean): 1.85 took: 4.01s
Epoch 49, 35% 	 train_loss(mean): 1.76 took: 3.91s
Epoch 49, 47% 	 train_loss(mean): 1.77 took: 4.13s
Epoch 49, 58% 	 train_loss(mean): 2.10 took: 5.70s
Epoch 49, 70% 	 train_loss(mean): 1.90 took: 3.92s
Epoch 49, 82% 	 train_loss(mean): 2.10 took: 4.32s
Epoch 49, 94% 	 train_loss(mean): 2.06 took: 4.48s
Start testing...
Accuracy 54.38%
...testing finished
total training loss for epoch 49: 15.561868
Epoch: 49
Epoch 50, 11% 	 train_loss(mean): 1.62 took: 3.95s
Epoch 50, 23% 	 train_loss(mean): 2.10 took: 4.13s
Epoch 50, 35% 	 train_loss(mean): 2.01 took: 5.49s
Epoch 50, 47% 	 train_loss(mean): 2.07 took: 4.03s
Epoch 50, 58% 	 train_loss(mean): 1.89 took: 4.10s
Epoch 50, 70% 	 train_loss(mean): 1.93 took: 4.88s
Epoch 50, 82% 	 train_loss(mean): 2.03 took: 4.16s
Epoch 50, 94% 	 train_loss(mean): 1.86 took: 3.97s
Start testing...
Accuracy 52.62%
...testing finished
total training loss for epoch 50: 15.506751
Epoch: 50
Epoch 51, 11% 	 train

Epoch 65, 23% 	 train_loss(mean): 2.14 took: 4.06s
Epoch 65, 35% 	 train_loss(mean): 1.81 took: 3.86s
Epoch 65, 47% 	 train_loss(mean): 2.25 took: 4.11s
Epoch 65, 58% 	 train_loss(mean): 1.87 took: 4.37s
Epoch 65, 70% 	 train_loss(mean): 1.86 took: 3.97s
Epoch 65, 82% 	 train_loss(mean): 1.87 took: 5.65s
Epoch 65, 94% 	 train_loss(mean): 1.69 took: 3.87s
Start testing...
Accuracy 53.38%
...testing finished
total training loss for epoch 65: 15.494387
Epoch: 65
Epoch 66, 11% 	 train_loss(mean): 2.02 took: 4.06s
Epoch 66, 23% 	 train_loss(mean): 2.11 took: 4.09s
Epoch 66, 35% 	 train_loss(mean): 1.75 took: 4.20s
Epoch 66, 47% 	 train_loss(mean): 2.25 took: 4.30s
Epoch 66, 58% 	 train_loss(mean): 1.79 took: 3.80s
Epoch 66, 70% 	 train_loss(mean): 1.79 took: 5.45s
Epoch 66, 82% 	 train_loss(mean): 1.71 took: 3.95s
Epoch 66, 94% 	 train_loss(mean): 2.11 took: 4.45s
Start testing...
Accuracy 53.50%
...testing finished
total training loss for epoch 66: 15.533006
Epoch: 66
Epoch 67, 11% 	 train

Epoch 81, 23% 	 train_loss(mean): 2.05 took: 4.22s
Epoch 81, 35% 	 train_loss(mean): 1.84 took: 3.97s
Epoch 81, 47% 	 train_loss(mean): 1.91 took: 4.33s
Epoch 81, 58% 	 train_loss(mean): 1.91 took: 4.02s
Epoch 81, 70% 	 train_loss(mean): 1.79 took: 4.14s
Epoch 81, 82% 	 train_loss(mean): 2.16 took: 4.11s
Epoch 81, 94% 	 train_loss(mean): 1.83 took: 3.80s
Start testing...
Accuracy 52.88%
...testing finished
total training loss for epoch 81: 15.520896
Epoch: 81
Epoch 82, 11% 	 train_loss(mean): 1.84 took: 3.88s
Epoch 82, 23% 	 train_loss(mean): 1.82 took: 4.13s
Epoch 82, 35% 	 train_loss(mean): 2.22 took: 4.25s
Epoch 82, 47% 	 train_loss(mean): 1.93 took: 4.02s
Epoch 82, 58% 	 train_loss(mean): 1.84 took: 3.96s
Epoch 82, 70% 	 train_loss(mean): 1.81 took: 4.12s
Epoch 82, 82% 	 train_loss(mean): 2.19 took: 5.35s
Epoch 82, 94% 	 train_loss(mean): 1.89 took: 4.18s
Start testing...
Accuracy 54.25%
...testing finished
total training loss for epoch 82: 15.534919
Epoch: 82
Epoch 83, 11% 	 train

Epoch 97, 23% 	 train_loss(mean): 1.69 took: 5.44s
Epoch 97, 35% 	 train_loss(mean): 1.94 took: 5.08s
Epoch 97, 47% 	 train_loss(mean): 1.97 took: 4.22s
Epoch 97, 58% 	 train_loss(mean): 2.04 took: 4.07s
Epoch 97, 70% 	 train_loss(mean): 1.91 took: 4.47s
Epoch 97, 82% 	 train_loss(mean): 1.81 took: 4.43s
Epoch 97, 94% 	 train_loss(mean): 1.91 took: 4.17s
Start testing...
Accuracy 53.38%
...testing finished
total training loss for epoch 97: 15.521572
Epoch: 97
Epoch 98, 11% 	 train_loss(mean): 2.03 took: 4.33s
Epoch 98, 23% 	 train_loss(mean): 1.77 took: 4.23s
Epoch 98, 35% 	 train_loss(mean): 2.01 took: 4.34s
Epoch 98, 47% 	 train_loss(mean): 2.08 took: 4.26s
Epoch 98, 58% 	 train_loss(mean): 1.59 took: 4.15s
Epoch 98, 70% 	 train_loss(mean): 1.84 took: 4.27s
Epoch 98, 82% 	 train_loss(mean): 1.82 took: 4.00s
Epoch 98, 94% 	 train_loss(mean): 2.35 took: 4.55s
Start testing...
Accuracy 54.12%
...testing finished
total training loss for epoch 98: 15.473736
Epoch: 98
Epoch 99, 11% 	 train

### training only last prediction

In [10]:
### define testing function
def test_last_pred(model):
    n_batches_test = len(test_loader)

    #Time for printing
    testing_start_time = time.time()

    print('Start testing...')
    correct = 0 
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            
            scenes = [s.to(device) for s in inputs['scenes']]
            labels = labels[:,-1].to(device)
            if not labels.size()[0] == BATCH_SIZE:
                # skip uncompleted batch size NN is fixed to BATCHSIZE
                continue
            outputs = model(scenes)
            outputs = outputs[:,:,-1]
#             print("Out:", len(outputs), outputs.size())
#             print("Labels:", len(labels), labels.size())
            _, predicted = torch.max(outputs.data, 1)
#             print('predicted:',len(predicted),predicted.size())
            n_errors = torch.nonzero(torch.abs(labels.long() - predicted)).size(0)
            total += predicted.numel()
            # print('predicted',predicted)
            correct += predicted.numel() - n_errors
            # print('labels',labels)
    print('Accuracy {:.2f}%'.format(100*correct/total))
    print('...testing finished')

In [ ]:
n_batches = len(train_loader)

#Time for printing
training_start_time = time.time()
    
print('Start training...')
for epoch in range(100): 
    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0
    
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        
        inputs, labels = batch
        scenes = [s.to(device) for s in inputs['scenes']]
        labels = labels[:,-1].to(device)
        if not labels.size()[0] == BATCH_SIZE:
            # skip uncompleted batch size NN is fixed to BATCHSIZE
            continue
        optimizer.zero_grad() 
        outputs = test_net(scenes)
        outputs = outputs[:,:,-1]
#         print("Out:", len(outputs), outputs.size())
#         print("Labels:", len(labels), labels.size())
        loss = criterion(outputs,labels.long() )
        loss.backward() 
        optimizer.step()
        
        #Print statistics
#         print('loss.data.item()', loss.data.item())
        running_loss += loss.data.item()
        total_train_loss += loss.data.item()
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss(mean): {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss/print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()
    # test after each epoch
    test_last_pred(test_net)
    print('total training loss for epoch {}: {:.6f}'.format(epoch+1, total_train_loss))

print('...Training finished')

Start training...
Epoch: 0


/home/ron/anaconda3/envs/gait_37/lib/python3.7/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


Epoch 1, 11% 	 train_loss(mean): 2.34 took: 5.20s
Epoch 1, 23% 	 train_loss(mean): 2.25 took: 4.46s
Epoch 1, 35% 	 train_loss(mean): 2.18 took: 4.48s
Epoch 1, 47% 	 train_loss(mean): 2.25 took: 4.62s
Epoch 1, 58% 	 train_loss(mean): 2.24 took: 5.99s
Epoch 1, 70% 	 train_loss(mean): 2.21 took: 4.57s
Epoch 1, 82% 	 train_loss(mean): 2.19 took: 5.16s
Epoch 1, 94% 	 train_loss(mean): 2.15 took: 4.57s
Start testing...
Accuracy 43.75%
...testing finished
total training loss for epoch 1: 17.813584
Epoch: 1
Epoch 2, 11% 	 train_loss(mean): 2.15 took: 4.56s
Epoch 2, 23% 	 train_loss(mean): 2.15 took: 5.11s
Epoch 2, 35% 	 train_loss(mean): 2.18 took: 4.92s
Epoch 2, 47% 	 train_loss(mean): 2.06 took: 4.13s
Epoch 2, 58% 	 train_loss(mean): 2.05 took: 5.00s
Epoch 2, 70% 	 train_loss(mean): 2.03 took: 6.30s
